In [7]:
import time
import re
import csv
import schedule
import shutil
import os
import unidecode
import pandas as pd
import preprocessor as p
from bs4 import BeautifulSoup
from pathlib import Path
from jmespath import search
from selenium import webdriver
from sklearn.feature_extraction.text import TfidfVectorizer

def removeDuplicates():
  lines_seen = set() # holds lines already seen
  outfile = open('documentosNoDuplicate.txt', "w")
  for line in open('documentos.txt', "r"):
    if line not in lines_seen: # not a duplicate
        outfile.write(line)
        lines_seen.add(line)
  outfile.close()

def idf():
  file = open("documentosNoDuplicate.txt","r", encoding="utf8")
  liststrings = file.readlines()
  liststrings.remove('Nao lidaBem-vindo ao Facebook! Toque aqui para encontrar pessoas que voce conhece e adiciona-las aos amigos.\n')

  tfIdfVectorizer=TfidfVectorizer(decode_error ='ignore',use_idf=True, lowercase=True)
  tf_idf = tfIdfVectorizer.fit_transform(liststrings)
  dataIdf = pd.DataFrame(tfIdfVectorizer.idf_ , index=tfIdfVectorizer.get_feature_names(), columns=['IDF']).T
  dataTf_Idf = pd.DataFrame(tf_idf.todense(), columns=tfIdfVectorizer.get_feature_names())

  for col in dataIdf.columns:
    if(col != 'Unnamed: 0'):
        dataTf_Idf.loc[dataTf_Idf[col] != 0.0, col] = dataIdf.iloc[0][col]

  dataTf_Idf.to_csv('idf.csv', sep=';', decimal=',', float_format='%.3f') 
  shutil.make_archive("idf", 'zip', os.path.expanduser('~') + "\Documents\Python\PJ4", "idf.csv")
  os.remove("idf.csv")



def saveScroll(url, driver, documentos):
      time.sleep(5) 

      driver.get(url)

      time.sleep(5) 

      SCROLL_PAUSE_TIME = 10

      # Get scroll height
      last_height = driver.execute_script("return document.body.scrollHeight")

      while True:
          # Scroll down to bottom
          driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

          # Wait to load page
          time.sleep(SCROLL_PAUSE_TIME)

          # Calculate new scroll height and compare with last scroll height
          new_height = driver.execute_script("return document.body.scrollHeight")
          if new_height == last_height:
              break
          last_height = new_height

      page_source = driver.page_source
      soup = BeautifulSoup(page_source, 'html.parser')
      reviews_selector = soup.find_all("span", {"class": "gvxzyvdx aeinzg81 t7p7dqev gh25dzvf exr7barw b6ax4al1 gem102v4 ncib64c9 mrvwc6qr sx8pxkcf f597kf1v cpcgwwas m2nijcs8 hxfwr5lz k1z55t6l oog5qr5w tes86rjd pbevjfx6 ztn2w49o"})
      
      for review_selector in reviews_selector:
          registro = re.sub(r"\d", "", review_selector.text)
          registro = registro.replace('... Ver mais',' ')
          registro = unidecode.unidecode(registro)
          documentos.write(p.clean(registro) + '\n')

def webScrapping():
  file1 = open(str(Path.home()) + "/Downloads/facepass.txt","r+")
  userpass = file1.readlines()
  user = userpass[0]
  passw = userpass[1]

  options = webdriver.ChromeOptions()
  options.add_argument("--enable-javascript")
  options.add_argument('--ignore-certificate-errors')
  options.add_argument('--incognito')
  options.add_argument('--headless')
  driver = webdriver.Chrome(options=options)


  driver.get('http://www.facebook.com');


  username = driver.find_element(webdriver.common.by.By.NAME,"email")
  password = driver.find_element(webdriver.common.by.By.NAME,"pass")
  submit   = driver.find_element(webdriver.common.by.By.NAME,"login")

  password.send_keys(passw)
  username.send_keys(user)

  time.sleep(5) 

  try:
    submit.click()
  except:
    print("Ignorar")

  documentos = open("documentos.txt", 'a+', newline='', encoding='utf-8')

  saveScroll('https://www.facebook.com/hashtag/ciro2022', driver, documentos)
  saveScroll('https://www.facebook.com/hashtag/bolsonaro2022', driver, documentos)
  saveScroll('https://www.facebook.com/hashtag/lula2022', driver, documentos)

  print("Terminado.")

  documentos.close()

  driver.quit()

  removeDuplicates()
  idf()

In [ ]:
schedule.every(1).days.do(webScrapping)

while 1:
  schedule.run_pending()
  time.sleep(100)